[![Banner-Modelos.png](https://i.postimg.cc/RCgy2XdL/Banner-Modelos.png)](https://postimg.cc/PN8SwbBC)

## L3:  Regresión regularizada. Caso Lasso.

El objetivo de este notebook es mostrar cómo construir un modelo de regresión con técnicas de regularización, con ajuste de hiperparámetros con técnicas de validación. En este caso utilizaremos regularización norma L1, que en el contexto de regresión se conoce como Lasso. También veremos cómo este tipo de regularización actúa como selector de variables en la regresión lineal.

Se explicará, paso a paso, como:

1. Preparar los datos para el modelado.
2. Utilizar técnicas de validación para el ajuste de hiperparámetros.
3. Construir un modelo con regresión Lasso.
4. Evaluar el modelo.
5. Determinar importancia de variables.

Se utiliza el conjunto de datos relacionado con la venta de vehículos. 

###### Autores: David Ocampo (d.ocampo@uniandes.edu.co), Julián Montoya (jc.montoyar@uniandes.edu.co)

### 1. Importación de librerías 

En las siguientes líneas de código se importan las librerías y herramientas necesarias para desarrollar el caso de uso.

In [1]:
# Librería para comando de sistema
import os
# Librerías para manejo de datos
import pandas as pd
# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
# Para la creación de modelo Ridge
from sklearn.linear_model import Lasso
# Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV
# Para la validación cruzada
from sklearn.model_selection import KFold 
# Para determinar el rendimiento del modelo con las métricas MSE, MAE y R2
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

### 2. Carga de los datos
A través de la librería **pandas** podemos realizar la carga de datos desde diferentes fuentes de información, en este caso se realizará la carga de un archivo plano csv (archivo separado por comas).

In [2]:
# Se cargan los datos. 
data=pd.read_csv('Datos_C2_Modelos_M3.csv',sep=';')

In [3]:
# Cantidad de datos y número de variables
data.shape

(8115, 9)

In [4]:
# Mostrar los datos
data.head()

,referencia,modelo,precio,kilometraje,combustible,propietario,motor,poder_maximo,asientos
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,First Owner,1248.0,74.00,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Second Owner,1498.0,103.52,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Third Owner,1497.0,78.00,5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,First Owner,1396.0,90.00,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,First Owner,1298.0,88.20,5.0


### 3. Limpieza y preparación de los datos

Recuerda que un aspecto muy importante para tener en cuenta son los requerimientos de entrada de los algoritmos de aprendizaje. Cada uno de estos puede trabajar con un tipo de variable, es por esto que vamos a realizar las mismas transformaciones que se realizaron en el notebook de regresión lineal. Además, vamos a ejecutar los mismos pasos de limpieza de los datos.

In [5]:
# Es recomendable que todos los pasos de preparación se realicen sobre otro archivo.
data_t = data
# Eliminación data vacía.
data_t=data_t.dropna()
# Eliminación de registros duplicados.
data_t=data_t.drop_duplicates()
# Transformación de los datos
data_t = pd.get_dummies(data_t, columns=['combustible','propietario'])
# Eliminación de la variable "referencia" del conjunto de datos.
data_t=data_t.drop(['referencia'], axis=1)

In [6]:
# Visualización de los datos limpios
data_t.head()

,modelo,precio,kilometraje,motor,poder_maximo,asientos,combustible_CNG,combustible_Diesel,combustible_LPG,combustible_Petrol,propietario_First Owner,propietario_Fourth & Above Owner,propietario_Second Owner,propietario_Third Owner
0,2014,450000,145500,1248.0,74.00,5.0,0,1,0,0,1,0,0,0
1,2014,370000,120000,1498.0,103.52,5.0,0,1,0,0,0,0,1,0
2,2006,158000,140000,1497.0,78.00,5.0,0,0,0,1,0,0,0,1
3,2010,225000,127000,1396.0,90.00,5.0,0,1,0,0,1,0,0,0
4,2007,130000,120000,1298.0,88.20,5.0,0,0,0,1,1,0,0,0


In [7]:
# El tamaño de los datos preparados
data_t.shape

(6707, 14)

### 4. Construcción del modelo

Los algoritmos supervisados implementados en scikit-learn requieren que las variables de entrada estén separadas de la variable objetivo. En este caso seguiremos con nuestra variable objetivo que es el precio. 

In [8]:
# Se selecciona la variable objetivo, en este caso "precio".
Y=data_t['precio']
# Del conjunto de datos se elimina la variable "precio".
X=data_t.drop(['precio'], axis=1)

In [9]:
# Se realiza la división entrenamiento - test. Se deja 20% de los datos para el test.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)

Como ya has podido conocer la regresión Lasso utiliza la norma L1 del vector de coeficientes como término de regularización. Para controlar la cantidad de regularización se utiliza el hiperparámetro alpha. Probemos con un valor igual a 1.

In [10]:
# Para acelerar la convergencia del algoritmo que utiliza Lasso para optimizar la función de costo, utilizaremos la opción
# de normalizar los datos para que todos estén en el mismo rango.
modelo_lasso = Lasso(alpha=1, normalize = 'True')
modelo_lasso

Lasso(alpha=1, normalize='True')

In [11]:
# Ajuste del modelo
modelo_lasso.fit(X_train,Y_train)

Lasso(alpha=1, normalize='True')

In [12]:
# Ahora probemos el rendimiento sobre el conjunto test.
y_pred = modelo_lasso.predict(X_test)
print("RMSE: %.2f" % mean_squared_error(Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))
print('R²: %.2f' % r2_score(Y_test, y_pred))

RMSE: 278449.84
MAE: 167830.13
R²: 0.61


Ahora vamos a utilizar la función GridSearchCV para la búsqueda del valor de alpha. 

In [13]:
# Fijemos el número de particiones. Utilizaremos K = 10.
particiones = KFold(n_splits=10, shuffle=True, random_state = 0)

In [14]:
# Ahora tenemos que definir el espacio de búsqueda, es decir, los valores de alpha que queremos que sean considerados. 
# Para esto se define un diccionario (o grilla) con los valores que podrá asumir el hiperparámetro alpha.
# Probemos con los siguientes valores:
param_grid = {'alpha': [1, 2, 3, 4, 5]}

In [15]:
# Definimos el modelo sin ningún valor del hiperparámetro alpha
modelo_lasso = Lasso(normalize = 'True')

In [16]:
# Ahora utilizamos GridSearch sobre el grid definido y con 10 particiones en la validación cruzada.
mejor_modelo = GridSearchCV(modelo_lasso, param_grid, cv=particiones, n_jobs=-1)
# Ajuste del modelo
mejor_modelo.fit(X_train, Y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
             estimator=Lasso(normalize='True'), n_jobs=-1,
             param_grid={'alpha': [1, 2, 3, 4, 5]})

In [17]:
# Podemos ver cual fue el resultado de la búsqueda (mejor valor de alpha)
print("Mejor parámetro: {}".format(mejor_modelo.best_params_)) 

Mejor parámetro: {'alpha': 5}


In [18]:
# También puedes indicarle a GridSearch que seleccione el mejor modelo a partir de la búsqueda con base en una métrica 
# particular. Por ejemplo, hubiésemos podido utilizar la siguiente línea de comando:
mejor_modelo = GridSearchCV(modelo_lasso, param_grid, scoring = 'neg_mean_absolute_error', n_jobs=-1 )
mejor_modelo.fit(X_train, Y_train)
print("Mejor parámetro: {}".format(mejor_modelo.best_params_)) 

Mejor parámetro: {'alpha': 5}


Un aspecto importante que debes tener en cuenta es que GridSearchCV devuelve el modelo entrenado con el mejor valor del hiperparámetro alpha (resultado de la búsqueda).

In [19]:
modelo_final = mejor_modelo.best_estimator_
# Probemos ahora este modelo sobre test.
y_pred = modelo_final.predict(X_test)
print("RMSE: %.2f" % mean_squared_error(Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))
print('R²: %.2f' % r2_score(Y_test, y_pred))

RMSE: 278426.01
MAE: 167702.65
R²: 0.61


### 5. Importancia de variables y selección

La regresión Lasso también actúa como un selector de variables: serán aquellas que tienen un coeficiente diferente de  cero. Entonces, con base en los valores de los coeficientes, podrás determinar la importancia de los atributos en el analisis y predicción. 

In [20]:
# Revisar los parámetros del modelo entrenado
coeficientes = modelo_final.coef_
variables = X_train.columns
# Mostrar en una tabla los valores de los coeficientes para cada variable
pd.DataFrame({'coeficientes':coeficientes,'variables':variables})

,coeficientes,variables
0,36304.851835,modelo
1,-0.732882,kilometraje
2,90.521178,motor
3,8879.175346,poder_maximo
4,-20646.520033,asientos
5,-0.000000,combustible_CNG
6,37446.451638,combustible_Diesel
7,104062.389932,combustible_LPG
8,-28303.131527,combustible_Petrol
9,61349.779714,propietario_First Owner


[![Banner-Modelos.png](https://i.postimg.cc/RCgy2XdL/Banner-Modelos.png)](https://postimg.cc/PN8SwbBC)